In [8]:
import pandas as pd

# Load the Excel file to inspect sheet names
file_path = "/content/Critical_Role_Campaign_1_Datapack.xlsx"
excel_file = pd.ExcelFile(file_path)

# Display sheet names to determine which ones to load
excel_file.sheet_names

['episode_details',
 'gameplay_times',
 'combat_times',
 'dialogue',
 'all_rolls',
 'sources']

In [9]:
dialogue_df = pd.read_excel(file_path, sheet_name='dialogue')

In [10]:
# Step 2: Sort the dataframe first
dialogue_df = dialogue_df.sort_values(by=['Episode', 'time_in_secs']).reset_index(drop=True)

# Create the end_time column BEFORE renaming anything
dialogue_df['end_time'] = dialogue_df.groupby('Episode')['time_in_secs'].shift(-1)

# Fill NaN in end_time with +5 seconds from time_in_secs
dialogue_df['end_time'] = dialogue_df['end_time'].fillna(dialogue_df['time_in_secs'] + 5)

# NOW rename the time_in_secs to start_time
dialogue_df = dialogue_df.rename(columns={'time_in_secs': 'start_time'})

# Optional: Calculate duration
dialogue_df['duration'] = dialogue_df['end_time'] - dialogue_df['start_time']

In [11]:
# Step 3: Flag where the speaker or episode changes (new group)
dialogue_df['speaker_change'] = (
    (dialogue_df['name'] != dialogue_df['name'].shift(1)) |
    (dialogue_df['Episode'] != dialogue_df['Episode'].shift(1))
).astype(int)

# Create a group ID that increments every time there's a change
dialogue_df['speech_group'] = dialogue_df['speaker_change'].cumsum()

# Group the data by group ID, Episode, and Speaker
grouped_df = dialogue_df.groupby(['speech_group', 'Episode', 'name']).agg({
    'start_time': 'min',
    'end_time': 'max',
    'dialogue': lambda x: ' '.join(str(d) for d in x if pd.notna(d))  # Combine text
}).reset_index()

# Recalculate duration for grouped speeches
grouped_df['duration'] = grouped_df['end_time'] - grouped_df['start_time']

In [14]:
all_rolls_df = pd.read_excel(file_path, sheet_name='all_rolls')
print(all_rolls_df.columns)

Index(['RollID', 'Campaign', 'Episode', 'Time', 'time_in_seconds',
       'youtube_timestamp', 'Player', 'Character', 'Type of Roll', 'D20?',
       'Total Roll', 'Natural Roll', 'Crit?', 'Notes'],
      dtype='object')


In [17]:
# Select only the relevant columns from all_rolls_df
mapping_df = all_rolls_df[['Player', 'Character']].drop_duplicates()

# Merge with grouped_df using name <-> Player
final_df = grouped_df.merge(mapping_df, left_on='name', right_on='Player', how='left')

In [18]:
# Drop the duplicate 'Player' column if needed
final_df = final_df.drop(columns=['Player'])

# Rename columns for Tableau clarity
final_df = final_df.rename(columns={
    'name': 'Speaker',
    'Character': 'Character'
})

# You can optionally assign a 'Role'
final_df['Role'] = final_df['Speaker'].apply(lambda x: 'DM' if x.upper() == 'MATT' else 'Player')

In [19]:
final_df.to_csv('/content/character_speech_blocks.csv', index=False)